<a href="https://colab.research.google.com/github/BillWorstell/Akan/blob/main/Ananse1/Ananse1Streamlit8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can create a simple interactive GUI in Python using Streamlit, a lightweight library that's perfect for deploying applications quickly, including on Google Colab. Streamlit will allow you to integrate text, images, and interact with GPT-4 API, all from a single interface with a friendly user experience.

Here’s how you can set up this interactive folklore exploration tool in Streamlit. The following code includes a GUI where users can explore the dilemmas, see paired stories, and interact with a GPT-4 API for deeper analysis or explanation of each tale. We’ll also add options for adaptive user control and seamless integration with GitHub for version tracking.

###Steps to Create and Deploy the Demo App
* Install Streamlit and other required libraries.
* Build the Streamlit GUI with options for choosing dilemmas and displaying the story pairs.
* Integrate GPT-4 API for analysis and interpretation.
* Deploy the app on Google Colab and save it to GitHub

#1. Install Streamlit and Required Libraries
In Google Colab, install Streamlit, requests (for API calls), and ngrok (for hosting the Streamlit app).

In [1]:
!pip install streamlit requests pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


###2. **Set Up the Ngrok Authentication Token**
Replace "your_authtoken" with your actual ngrok token in the following cell.

In [2]:
#!ngrok config add-authtoken "your_authtoken"
#!ngrok authtoken "your_authtoken"
!ngrok config add-authtoken "2ol9K0GNdjTzB2GrP2i2LES3WVu_EDz1dLjEE5YJ2yMjXAob"
!ngrok authtoken "2ol9K0GNdjTzB2GrP2i2LES3WVu_EDz1dLjEE5YJ2yMjXAob"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


###3. **Mount Google drive for Excel Databases**
Mount to /content/drive

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

###4. **Save Your Streamlit App Code in app.py**
In a new Colab cell, copy your Streamlit app code and save it as app.py:

In [4]:
code = """
# Import required libraries
# Import required libraries
import pandas as pd
import streamlit as st

# Define styling rules based on POS tags
styling_rules = {
    "NOUN": {"color": "blue", "bold": True},       # Nouns
    "PRON": {"color": "purple", "italic": True},   # Pronouns
    "VERB": {"color": "green", "underline": True}, # Verbs
    "ADV": {"color": "orange"},                   # Adverbs
    "CONJ": {"color": "red", "bold": True},        # Conjunctions
    "ADP": {"color": "cyan"},                     # Prepositions
    "DET": {"color": "gray", "italic": True},     # Articles
    "PUNCT": {"color": "black"},                 # Punctuation
    "PROPN": {"color": "darkblue", "bold": True, "italic": True}, # Proper Nouns
    "INTJ": {"color": "pink", "italic": True},    # Interjections
    "NUM": {"color": "teal"},                     # Numbers
    "PART": {"color": "brown"},                   # Particles
    "SCONJ": {"color": "darkred"},                # Subordinating conjunctions
    "CCONJ": {"color": "lightcoral"},             # Coordinating conjunctions
    "ADJ": {"color": "magenta", "italic": True},  # Adjectives
    "TEXT": {"color": "brown", "font_name": "Courier New"} # Direct Speech
}

# Function to style a sentence using POS tags# Function to style a sentence using POS tags

def style_sentence_with_pos(sentence, pos_tags):

    #Styles a sentence using POS tags and styling rules.
    #:param sentence: Original sentence as a string.
    #:param pos_tags: POS tags as a list of strings in the format <word|POS>.
    #:return: Styled HTML string.

    styled_words = []
    for word_pos in pos_tags:
        try:
            # Check for proper formatting and split word and POS
            if "|" in word_pos and word_pos.startswith("<") and word_pos.endswith(">"):
                word, pos = word_pos.strip('<>').rsplit('|', 1)
            else:
                # If no POS tag, treat as punctuation or unknown
                word = word_pos.strip('<>')
                pos = "PUNCT" if word in [".", ",", "!", "?", ";", ":"] else "UNKNOWN"

            # Remove any residual "ket" format from the word
            if "|" in word:
                word = word.split('|')[0]  # Keep only the actual word part

            # Get style based on POS
            style = styling_rules.get(pos, {"color": "black"})  # Default to black
            color = f"color:{style.get('color', 'black')};"
            bold = "font-weight:bold;" if style.get("bold", False) else ""
            italic = "font-style:italic;" if style.get("italic", False) else ""
            underline = "text-decoration:underline;" if style.get("underline", False) else ""
            font_name = f"font-family:{style.get('font_name', 'Arial')};"

            # Avoid styling punctuation
            if pos == "PUNCT":
                styled_word = word  # Render punctuation as is
            else:
                styled_word = f'<span style="{color} {bold} {italic} {underline} {font_name}">{word}</span>'
            styled_words.append(styled_word)
        except Exception as e:
            # Handle malformed tags gracefully
            styled_words.append(f'<span style="color:red;">{word_pos}</span>')  # Highlight problematic tags

    return ' '.join(styled_words)


# Streamlit app setup
st.title("HOW KWAKU ANANSE (THE SPIDER) GOT ASO IN MARRIAGE")

# URL of the raw Excel file in the GitHub repository
url_Story = "https://raw.githubusercontent.com/BillWorstell/Akan/main/Ananse1/Ananse1.xlsx"

# Load the Excel file
story_file = pd.ExcelFile(url_Story, engine="openpyxl")

# Sidebar integration
st.sidebar.header("Story= Ananse1")

# Story levels and sentence selector
story_levels = story_file.sheet_names
level_choice = st.sidebar.selectbox("Story Level", story_levels)
story_data = story_file.parse(level_choice)
num_sentences = len(story_data)
sentence = st.sidebar.slider("Sentence Number?", 0, num_sentences - 1, 0)

# Main section split into four columns
col1, col2, col3, col4= st.columns(4)

# Populate Column 1 (Akan)
with col1:
    st.write("### Akan")
    akan_sentence = story_data.iloc[sentence, 0]
    akan_pos_tags = story_data.iloc[sentence, 4].split()  # Assumes POS tags are in the 5th column
    styled_akan = style_sentence_with_pos(akan_sentence, akan_pos_tags)
    st.markdown(styled_akan, unsafe_allow_html=True)

# Populate Column 2 (AtoE)
with col2:
    st.write("### AtoE")
    atoe_sentence = story_data.iloc[sentence, 2]
    atoe_pos_tags = story_data.iloc[sentence, 6].split()  # Assumes POS tags are in the 7th column
    styled_atoe = style_sentence_with_pos(atoe_sentence, atoe_pos_tags)
    st.markdown(styled_atoe, unsafe_allow_html=True)

# Populate Column 3 (English)
with col3:
    st.write("### English")
    english_sentence = story_data.iloc[sentence, 1]
    english_pos_tags = story_data.iloc[sentence, 5].split()  # Assumes POS tags are in the 6th column
    styled_english = style_sentence_with_pos(english_sentence, english_pos_tags)
    st.markdown(styled_english, unsafe_allow_html=True)

# Populate Column 4 (EtoA)
with col4:
    st.write("### EtoA")
    etoa_sentence = story_data.iloc[sentence, 3]
    etoa_pos_tags = story_data.iloc[sentence, 7].split()  # Assumes POS tags are in the 8th column
    styled_etoa = style_sentence_with_pos(etoa_sentence, etoa_pos_tags)
    st.markdown(styled_etoa, unsafe_allow_html=True)


st.write("Enjoy folklore!")
"""

with open("app.py", "w") as file:
    file.write(code)

###Key Features:
1. Independent Level Selection for Both Columns:

* Each column (Column 1 and Column 2) has its own independent drop-downs in the sidebar.
* Users can select different levels of Akan, AtoE, and English for comparison.
2. Side-by-Side Display:

* Both columns display the selected levels for Akan, AtoE, and English texts.
* Useful for comparing different translation levels.
3. Sidebar Organization:

* Clearly separates settings for Column 1 and Column 2 to make selections intuitive.
###Expected Output:
* Sidebar: Allows level selection for both columns independently.
* Main Display:
** Column 1: Displays Akan, AtoE, and English texts as per Column 1 selections.
** Column 2: Displays Akan, AtoE, and English texts as per Column 2 selections.

In [5]:
#!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py

### 4. **Run the Streamlit App and Create the Ngrok Tunnel**
Now, run the following code to start the Streamlit app and create an ngrok tunnel to access it. Make sure only one instance of Streamlit is running to avoid conflicts.

In [6]:
import subprocess
# Start Streamlit app on port 8501
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])


<Popen: returncode: None args: ['streamlit', 'run', 'app.py', '--server.port...>

In [7]:
from pyngrok import ngrok

# Start a single ngrok tunnel to port 8501
url = ngrok.connect(8501)
print(f"Access the Streamlit app at: {url}")


Access the Streamlit app at: NgrokTunnel: "https://2573-34-16-202-98.ngrok-free.app" -> "http://localhost:8501"


###**Step 5: Confirm Streamlit is Running on Port 8501**
To verify that the Streamlit app is accessible, run:

In [8]:
!curl -I http://localhost:8501


curl: (7) Failed to connect to localhost port 8501 after 0 ms: Connection refused


If Streamlit is running correctly, this command should show a response. If there’s an error, check if the Streamlit app has started correctly in the cell above.

###**Important Notes:**

*   Avoid running multiple cells that initiate ngrok simultaneously, as this will exceed the session limit.
*   If you still face issues, go to the ngrok dashboard to terminate any active sessions before starting a new one in Colab.


.